In [24]:
% matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, Birch, DBSCAN



genres = ["Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
# ratings = pd.read_csv("data/ratings.csv")
# movies = pd.read_csv("data/movies.csv")
ratings = pd.read_csv("data/ratings.dat", sep="::", names=["userId", "movieId", "rating", "timestamp"])
movies = pd.read_csv("data/movies.dat", sep="::", names=["movieId", "title", "genres"])

/home/anna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/home/anna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


# Macierz użyteczności
rzędy = użytkownicy, 
kolumy = filmy

Może się okazać, że będzie trzeba zastosować inny rodzaj sparse_matrix

In [28]:
utility_matrix = coo_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))
utility_matrix.toarray()[:10, :10]
# movies.head()
# ratings.head()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 5, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 4, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
       [0, 4, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 5, 0, 0, 0, 0, 0, 0, 0, 0]])

In [19]:
for i, v in enumerate(genres):
    movies[i] = movies['genres'].str.contains(v)
# tags = pd.read_csv("data/tags.csv")

movies = movies.merge(ratings.groupby('movieId')['rating'].agg([pd.np.mean]), how='left', on='movieId')
movies.head()
# ratings.head()

,movieId,title,genres,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,mean_x,mean_y
0,1,Toy Story (1995),Animation|Children's|Comedy,False,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,4.146846,4.146846
1,2,Jumanji (1995),Adventure|Children's|Fantasy,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,3.201141,3.201141
2,3,Grumpier Old Men (1995),Comedy|Romance,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,3.016736,3.016736
3,4,Waiting to Exhale (1995),Comedy|Drama,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,2.729412,2.729412
4,5,Father of the Bride Part II (1995),Comedy,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,3.006757,3.006757


In [3]:
ratingsExt = pd.merge(ratings, movies, on='movieId')
ratingsExt.head()

KeyError: 'movieId'

In [ ]:
for i in range(18):
    ratingsExt[i] = ratingsExt[i] * (ratingsExt['rating'] - ratingsExt['mean']) 

ratingsExt.head()


In [ ]:
ratingsExt.groupby('userId')[list(range(18))].agg([pd.np.sum]).head()

In [ ]:
data = ratingsExt.drop(["timestamp", "title", "genres"], axis = 1)
data.head()

## Przykłądowe dane:

In [ ]:
ratingsExt.loc[(ratingsExt['userId'] == 1) & (ratingsExt['rating'] > 4)]

## Próba grupowania filmów według gatunków (w seumie bez sensu)

In [ ]:

test = movies.drop(["movieId", 'genres'], axis = 1)

data = test.loc[:, range(18)]
print(data.shape)

pca_model = PCA(n_components=2)
pca = pca_model.fit_transform(data)

kmeans = KMeans(n_clusters=4, init='k-means++', n_init=100).fit(data)
birch = Birch(threshold=0.2, n_clusters=None).fit(data)
dbscan = DBSCAN(eps=0.999999, min_samples=25).fit(data)

In [ ]:
x, y = pca[:,0], pca[:,1]


plt.axis('equal')
plt.scatter(x,y, c=kmeans.labels_)
plt.show()

plt.axis('equal')
plt.scatter(x,y, c=birch.labels_)
plt.show()

plt.axis('equal')
plt.scatter(x,y, c=dbscan.labels_)
plt.show()